In [1]:
import pandas as pd
from functools import reduce
import numpy as np

In [2]:
s_raw = """marathon: Big_Jim 2nd_astronaut RidewiththeFlow südpfälzer All_mtn Spoon-man mogg AndySch dino113
xc: Dirty Track Big_Jim A-Abraxas 2nd_astronaut RidewiththeFlow südpfälzer Diddo DeluXer Mr.Hahn rhnordpool All_mtn Tony- PanicMen Spoon-man mogg Mountain_4Life AndySch Schoenpflug stummerwinter Anferd Rubin HazelNoos dino113 Ritzelabzieher Daleo
gravel: MikeyBaloooooza A-Abraxas RidewiththeFlow südpfälzer discordius jalgrattad Bene2405 Tony- TiCiLA AndySch stummerwinter dino113 Daleo
enduro: Suicide loeffel MikeyBaloooooza mad_raven Duefi cxfahrer mäxx__ Walkerk EagleEye Drahtacus Preme Ghostrider72 x_FreiRider_x Lambutz DeluXer Mr.Hahn boescha All_mtn Bene2405 pointidani ElDiabolo666 Tony- ChaosSpeed31 Schoenpflug luisuet1 Anferd nanananaMADMAN k0p3 SwishSwushPow bikerniki
DH: loeffel MikeyBaloooooza mad_raven mäxx__ EagleEye Votec Tox Drahtacus x_FreiRider_x Lambutz Zask06 Bene2405 ChaosSpeed31 k0p3 SwishSwushPow
dirt: MikeyBaloooooza mad_raven mäxx__ nanananaMADMAN
ba: Zask06
street: MikeyBaloooooza mad_raven mäxx__ südpfälzer Mr.Hahn Rubin HazelNoos
rr: MikeyBaloooooza A-Abraxas EagleEye südpfälzer discordius boescha AndySch Schoenpflug Rubin HazelNoos
bmx: Bene2405
rt: MikeyBaloooooza discordius Schoenpflug"""

In [3]:
s_arr = s_raw.lower().split('\n')
s_keys = list(map(lambda x: x.split(': ')[0], s_arr))
s_val = list(map(lambda x: list(filter(bool, x.split(': ')[1].split(" "))), s_arr))

In [4]:
lens = list(map(len, s_val))
N = len(lens)
unq, col = np.unique(np.concatenate(s_val),return_inverse=1)
row = np.repeat(np.arange(N), lens)

In [5]:
out = np.zeros((N,len(unq)),dtype=int)
out[row,col] = 1

In [6]:
df = pd.DataFrame(out, columns=unq, index=s_keys).transpose()

In [7]:
df.corr()

,marathon,xc,gravel,enduro,dh,dirt,ba,street,rr,bmx,rt
marathon,1.000000,0.470052,0.220043,-0.372560,-0.264683,-0.121367,-0.059005,-0.018377,0.049874,-0.059005,-0.104111
xc,0.470052,1.000000,0.166586,-0.569231,-0.563093,-0.258199,-0.125529,0.081200,0.126878,-0.125529,-0.062471
gravel,0.220043,0.166586,1.000000,-0.336202,-0.141558,0.011731,-0.074141,0.047959,0.295806,0.245234,0.244860
enduro,-0.372560,-0.569231,-0.336202,1.000000,0.320530,0.258199,-0.144841,0.027067,-0.126878,0.125529,0.062471
dh,-0.264683,-0.563093,-0.141558,0.320530,1.000000,0.301964,0.222928,0.137169,-0.071444,0.222928,0.035177
dirt,-0.121367,-0.258199,0.011731,0.258199,0.301964,1.000000,-0.037398,0.524142,0.051726,-0.037398,0.241948
ba,-0.059005,-0.125529,-0.074141,-0.144841,0.222928,-0.037398,1.000000,-0.050965,-0.062869,-0.018182,-0.032081
street,-0.018377,0.081200,0.047959,0.027067,0.137169,0.524142,-0.050965,1.000000,0.387699,-0.050965,0.149873
rr,0.049874,0.126878,0.295806,-0.126878,-0.071444,0.051726,-0.062869,0.387699,1.000000,-0.062869,0.510272
bmx,-0.059005,-0.125529,0.245234,0.125529,0.222928,-0.037398,-0.018182,-0.050965,-0.062869,1.000000,-0.032081


In [8]:
def cond_prob(df, col_idx):
    col = df.iloc[:, col_idx]
    col_total = col.sum()
    cond_values = df.transpose().mul(col.values).transpose().sum()
    probs = cond_values / col_total
    return probs

In [9]:
cols = df.shape[1]

In [10]:
cond_probs = pd.DataFrame([cond_prob(df, idx) for idx in range(cols)], columns=df.transpose().index, index=df.transpose().index)
cond_probs

,marathon,xc,gravel,enduro,dh,dirt,ba,street,rr,bmx,rt
marathon,1.000000,1.000000,0.444444,0.111111,0.000000,0.000000,0.000000,0.111111,0.222222,0.000000,0.000000
xc,0.346154,1.000000,0.307692,0.230769,0.000000,0.000000,0.000000,0.153846,0.230769,0.000000,0.038462
gravel,0.307692,0.615385,1.000000,0.230769,0.153846,0.076923,0.000000,0.153846,0.384615,0.076923,0.153846
enduro,0.033333,0.200000,0.100000,1.000000,0.400000,0.133333,0.000000,0.133333,0.133333,0.033333,0.066667
dh,0.000000,0.000000,0.133333,0.800000,1.000000,0.200000,0.066667,0.200000,0.133333,0.066667,0.066667
dirt,0.000000,0.000000,0.250000,1.000000,0.750000,1.000000,0.000000,0.750000,0.250000,0.000000,0.250000
ba,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
street,0.142857,0.571429,0.285714,0.571429,0.428571,0.428571,0.000000,1.000000,0.571429,0.000000,0.142857
rr,0.200000,0.600000,0.500000,0.400000,0.200000,0.100000,0.000000,0.400000,1.000000,0.000000,0.300000
bmx,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
